In [15]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')


In [16]:
import ast
from owlready2 import *


class Visitor(ast.NodeVisitor):

    def __init__(self, onto):
        self.onto = onto

    def generic_visit(self, node):
        ast.NodeVisitor.generic_visit(self, node)
        with self.onto as onto:
            if type(node) == ast.ClassDef:
                for obj in node.bases:
                    if obj.id == "Node":
                        types.new_class(node.name, (Thing,))
                    else:
                        types.new_class(node.name, (onto[obj.id],))

            elif type(node) == ast.Assign:
                for el in node.value.elts:
                    if el.s == "body" or el.s == "parameters":
                        types.new_class(el.s, (ObjectProperty,))
                    else:
                        types.new_class(
                            "jname" if el.s == "name" else el.s,
                            (DataProperty,))


In [17]:
import ast
import sys

from os import path as Path
from owlready2 import get_ontology

ontology_path = './computed/tree.owl'

def create_ontology(path):
    with open(path, "r") as source:
        tree = ast.parse(source.read())

    ontology = get_ontology("http://Java_Ontology/tree.owl")

    visitor = Visitor(ontology)
    visitor.visit(tree)
    
    ontology.save(ontology_path, format="rdfxml")

    print('Ontology created ==> "%s" ' % ontology_path)

In [18]:
create_ontology("./resources/tree.py")

Ontology created ==> "./computed/tree.owl" 


In [19]:
onto = get_ontology(ontology_path).load()
cd = onto["ClassDeclaration"]

assert cd.name == "ClassDeclaration"
assert len(cd.is_a) == 1
assert cd.is_a[0].name == 'TypeDeclaration'